Mario Jimenez;  David Seijas;   Alejandro Ramirez

# Práctica 1C - Inteligencia Artificial
## Belén Díaz Agudo -  Facultad de Informática UCM
## Búsqueda local
En esta primera parte usaremos ejercicios paso a paso para familiarizarnos con la resolución de problemas sencillos de optimización, como la maximización o minimización de una función, o el problema de la mochila o del viajante, problemas conocidos cuya resolución se ha abordado con técnicas algorítmicas y que vamos a resolver utilizando algoritmos de búsqueda local. En la segunda parte de la práctica se pide resolver el problema de la organización de jornadas informáticas dado en el enunciado.

## Parte 1. Algoritmo de escalada
Hill Climbing es un algoritmo de búsqueda local heurística utilizada para problemas de optimización.
Esta solución puede o no ser el óptimo global. El algoritmo es una variante del algoritmo de generación y prueba.
<br>
En general, el algoritmo funciona de la siguiente manera:
- Evaluar el estado inicial.
- Si es igual al estado del objetivo, terminamos.
- Encuentra un estado vecino al estado actual
- Evaluar este estado. Si está más cerca del estado objetivo que antes, reemplace el estado inicial con este estado y repita estos pasos.
<br>
Usaremos la implementación de AIMA que está en el módulo search.py

    def hill_climbing(problem):
        """From the initial node, keep choosing the neighbor with highest value,
        stopping when no neighbor is better. [Figure 4.2]"""
        current = Node(problem.initial)
        while True:
            neighbors = current.expand(problem)
            if not neighbors:
                break
            neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
            if problem.value(neighbor.state) <= problem.value(current.state):
                break
            current = neighbor
        return current.state


### TSP (Travelling Salesman Problem): el problema del viajante
Dado un conjunto de ciudades y la distancia entre cada par de ciudades, el problema es encontrar la ruta más corta posible que visite cada ciudad exactamente una vez y regrese al punto de partida. Es un problema NP hard. No existen una solución de coste polinomial. 

In [13]:
cd aima-python

C:\Users\aleja\Google Drive\4o\INFORMATICA\INTELIGENCIA ARTIFICIAL\PRACTICAS\aima-python


In [14]:
from search import *


In [15]:
##Resolvereremos el problema del viajante TSP para encontrar una solución aproximada.

class TSP_problem(Problem):

    def two_opt(self, state):
        """ Neighbour generating function for Traveling Salesman Problem """
        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state) - 1)
        right = random.randint(0, len(neighbour_state) - 1)
        if left > right:
            left, right = right, left
        neighbour_state[left: right + 1] = reversed(neighbour_state[left: right + 1])
        return neighbour_state

    def actions(self, state):
        """ action that can be excuted in given state """
        return [self.two_opt]

    def result(self, state, action):
        """  result after applying the given action on the given state """
        return action(state)

    def path_cost(self, c, state1, action, state2):
        """ total distance for the Traveling Salesman to be covered if in state2  """
        cost = 0
        for i in range(len(state2) - 1):
            cost += distances[state2[i]][state2[i + 1]]
        cost += distances[state2[0]][state2[-1]]
        return cost

    def value(self, state):
        """ value of path cost given negative for the given state """
        return -1 * self.path_cost(None, None, None, state)

In [16]:
## Resolveremos el TSP para las ciudades de la lista de ciudades de Rumanía.
## ['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']

In [17]:
# Usaremos la siguiente representacion del libro AIMA para el mapa de Rumanía.

romania_map = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

romania_map.locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

Es bastante sencillo entender este `romania_map`. El primer nodo ** Arad ** tiene tres vecinos llamados ** Zerind **, ** Sibiu **, ** Timisoara **. Cada uno de estos nodos son 75, 140, 118 unidades aparte de ** Arad ** respectivamente. Y lo mismo ocurre con otros nodos.

Y `romania_map.locations` contiene las posiciones de cada uno de los nodos. 
Como heurística se puede usar la distancia en línea recta o la distancia manhattan (que es diferente de la proporcionada en `romania_map`) entre dos ciudades.

In [18]:
romania_locations = romania_map.locations
print(romania_locations)

{'Arad': (91, 492), 'Bucharest': (400, 327), 'Craiova': (253, 288), 'Drobeta': (165, 299), 'Eforie': (562, 293), 'Fagaras': (305, 449), 'Giurgiu': (375, 270), 'Hirsova': (534, 350), 'Iasi': (473, 506), 'Lugoj': (165, 379), 'Mehadia': (168, 339), 'Neamt': (406, 537), 'Oradea': (131, 571), 'Pitesti': (320, 368), 'Rimnicu': (233, 410), 'Sibiu': (207, 457), 'Timisoara': (94, 410), 'Urziceni': (456, 350), 'Vaslui': (509, 444), 'Zerind': (108, 531)}


In [19]:
# node colors, node positions and node label positions
node_colors = {node: 'white' for node in romania_map.locations.keys()}
node_positions = romania_map.locations
node_label_pos = { k:[v[0],v[1]-10]  for k,v in romania_map.locations.items() }
edge_weights = {(k, k2) : v2 for k, v in romania_map.graph_dict.items() for k2, v2 in v.items()}

romania_graph_data = {  'graph_dict' : romania_map.graph_dict,
                        'node_colors': node_colors,
                        'node_positions': node_positions,
                        'node_label_positions': node_label_pos,
                         'edge_weights': edge_weights
                     }

In [20]:
'''from notebook import show_map, final_path_colors, display_visual
import matplotlib

show_map(romania_graph_data)
'''

'from notebook import show_map, final_path_colors, display_visual\nimport matplotlib\n\nshow_map(romania_graph_data)\n'

In [21]:
## el siguiente código crea un diccionario y calcula y añade al diccionario la distancia manhattan entre las ciudades. 
import numpy as np

distances = {}
all_cities = []

for city in romania_map.locations.keys():
    distances[city] = {}
    all_cities.append(city)
    
all_cities.sort()
print(all_cities)

for name_1, coordinates_1 in romania_map.locations.items():
        for name_2, coordinates_2 in romania_map.locations.items():
            distances[name_1][name_2] = np.linalg.norm(
                [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])
            distances[name_2][name_1] = np.linalg.norm(
                [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])

['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']


In [22]:
# Creamos una instancia del problema TSP con la lista de ciudades anterior que se na extraido del mapa.
# En el mapa hay informacion de las distancias que se utilizan en la clase TSP_problem para calcular el coste y las heurísticas.
tsp = TSP_problem(all_cities)

In [23]:
## Redefinimos el hill climbing de AIMA para que el método de generacion de vecinos sea acceder al grafo que hemos definido para el TSP

def hill_climbing(problem):
    
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    
    def find_neighbors(state, number_of_neighbors=100):
        """ finds neighbors using two_opt method """
        
        neighbors = []
        
        for i in range(number_of_neighbors):
            new_state = problem.two_opt(state)
            neighbors.append(Node(new_state))
            state = new_state
            
        return neighbors

    # as this is a stochastic algorithm, we will set a cap on the number of iterations
    iterations = 10000
    
    current = Node(problem.initial)
    while iterations:
        neighbors = find_neighbors(current.state)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) >= problem.value(current.state):
            current.state = neighbor.state
        iterations -= 1
        
    return current.state

In [24]:
hill_climbing(tsp)

['Fagaras',
 'Neamt',
 'Iasi',
 'Vaslui',
 'Hirsova',
 'Eforie',
 'Urziceni',
 'Bucharest',
 'Giurgiu',
 'Pitesti',
 'Craiova',
 'Drobeta',
 'Mehadia',
 'Lugoj',
 'Timisoara',
 'Arad',
 'Zerind',
 'Oradea',
 'Sibiu',
 'Rimnicu']

In [25]:
hill_climbing(tsp)

['Arad',
 'Timisoara',
 'Lugoj',
 'Mehadia',
 'Drobeta',
 'Craiova',
 'Pitesti',
 'Giurgiu',
 'Bucharest',
 'Urziceni',
 'Eforie',
 'Hirsova',
 'Vaslui',
 'Iasi',
 'Neamt',
 'Fagaras',
 'Rimnicu',
 'Sibiu',
 'Oradea',
 'Zerind']

In [26]:
hill_climbing(tsp)

['Lugoj',
 'Mehadia',
 'Drobeta',
 'Craiova',
 'Pitesti',
 'Giurgiu',
 'Bucharest',
 'Urziceni',
 'Eforie',
 'Hirsova',
 'Vaslui',
 'Iasi',
 'Neamt',
 'Fagaras',
 'Rimnicu',
 'Sibiu',
 'Oradea',
 'Zerind',
 'Arad',
 'Timisoara']

### Ejercicio 1. Resuelve el problema TSP con el algoritmo de escalada por máxima pendiente en el mapa de ciudades de Rumanía y explica el resultado obtenido. 

Realiza un análisis razonado de las propiedades del algoritmo: eficiencia y optimalidad en base a la ejecución. 

¿Ha encontrado el algoritmo el óptimo global? 
¿Ha encontrado la misma solución en distintas ejecuciones?

Sólo se pide hacer una comparativa teórica (breve) con cómo se comporta este algoritmo y relacionarlo con otros algoritmos vistos en clase. 

Opcionalmente se puede hacer la comparativa real con algún algoritmo de búsqueda exhaustiva. 

1. En teoría no tiene por qué encontrar el óptimo global. Esto se debe a que el algoritmo puede estancarse en óptimos locales, suceso que se da cuando los vecinos de un estado son peores que el propio estado.
2. No encuentra la misma solución en distintas ejecuciones, pues el algoritmo no es determinista. En la generación de los vecinos interviene la aleatoriedad. Por ello, para un mismo estado y diferentes iteraciones, se generan distintos vecinos.
3. A la hora de encontrar el óptimo global, el algoritmo del enfriamiento simulado tiene más posibilidad, ya que permite dirigirse hacia un vecino peor con una probabilidad que decrece con cada iteración. De forma abstracta, esto consigue evitar estancarse en óptimos locales.


## Parte 2. Enfriamiento simulado ( simulated annealing) 
El algoritmo de enfriamiento simulado puede manejar las situaciones de óptimo local o mesetas típicas en algoritmos de escalada.
<br>
El enfriamiento simulado es bastante similar a la escalada pero en lugar de elegir el mejor movimiento en cada iteración, elige un movimiento aleatorio. Si este movimiento aleatorio nos acerca al óptimo global, será aceptado,
pero si no lo hace, el algoritmo puede aceptar o rechazar el movimiento en función de una probabilidad dictada por la temperatura.  Cuando la `temperatura` es alta, es más probable que el algoritmo acepte un movimiento aleatorio incluso si es malo. A bajas temperaturas, solo se aceptan buenos movimientos, con alguna excepción ocasional.
Esto permite la exploración del espacio de estado y evita que el algoritmo se atasque en el óptimo local.

    Usaremos la implementación de AIMA del modulo search.py
    
    def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice

Como hemos visto en clase hay varios métodos de enfriamiento (scheduling routine) 
Se puede variar el método de enfriamiento. En la implementación actual estamos usando el método de enfriamiento exponencial (que se pasa como parámetro). 

    def exp_schedule(k=20, lam=0.005, limit=100):
        """One possible schedule function for simulated annealing"""
        return lambda t: (k * math.exp(-lam * t) if t < limit else 0)

Como ejemplo, vamos a definir un problema sencillo de encontrar el punto más alto en una rejilla. Este problema está definido en el módulo search.py como PeakFindingProblem. Lo reproducimos aquí y creamos una rejilla simple.

In [27]:
initial = (0, 0)
grid = [[3, 7, 2, 8], [5, 2, 9, 1], [5, 3, 3, 1]]

In [28]:
# Pre-defined actions for PeakFindingProblem
directions4 = { 'W':(-1, 0), 'N':(0, 1), 'E':(1, 0), 'S':(0, -1) }
directions8 = dict(directions4) 
directions8.update({'NW':(-1, 1), 'NE':(1, 1), 'SE':(1, -1), 'SW':(-1, -1) })

class PeakFindingProblem(Problem):
    """Problem of finding the highest peak in a limited grid"""

    def __init__(self, initial, grid, defined_actions=directions4):
        """The grid is a 2 dimensional array/list whose state is specified by tuple of indices"""
        Problem.__init__(self, initial)
        self.grid = grid
        self.defined_actions = defined_actions
        self.n = len(grid)
        assert self.n > 0
        self.m = len(grid[0])
        assert self.m > 0

    def actions(self, state):
        """Returns the list of actions which are allowed to be taken from the given state"""
        allowed_actions = []
        for action in self.defined_actions:
            next_state = vector_add(state, self.defined_actions[action])
            if next_state[0] >= 0 and next_state[1] >= 0 and next_state[0] <= self.n - 1 and next_state[1] <= self.m - 1:
                allowed_actions.append(action)

        return allowed_actions

    def result(self, state, action):
        """Moves in the direction specified by action"""
        return vector_add(state, self.defined_actions[action])

    def value(self, state):
        """Value of a state is the value it is the index to"""
        x, y = state
        assert 0 <= x < self.n
        assert 0 <= y < self.m
        return self.grid[x][y]


In [29]:
problem = PeakFindingProblem(initial, grid, directions4)

In [30]:
# Lo resolvemos con enfriamiento simulado

solutions = [problem.value(simulated_annealing(problem)) for i in range(100)]
count=0
for i in range(100):
    if solutions[i]>=7:
        count+=1
print(max(solutions))
count

9


41

In [31]:
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state

In [32]:
solution = problem.value(hill_climbing(problem))
solution

7

### Ejercicio 2.  Resuelve el problema anterior de encontrar el punto máximo en una rejilla. Comenta y razona los resultados obtenidos en distintas rejjillas con los algoritmos de enfriamiento simulado y escalada por máxima pendiente. 
 
 
Ejemplo de rejilla para pruebas

grid = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 11.2, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]


In [33]:
initial = (0, 0)
grid =  [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 11.2, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]
grid2 = [[5.00, -1.00, 3.00, 1.00, 3.00, 4.00, -4.00],
        [4.00, -1.00, -4.00, -2.00,3.00, -2.00, 2.00],
        [-2.00, 2.00, 4.00, 12.20, 8.40, 5.00, -1.00],
        [-6.00, -2.00, -3.00,2.00, 1.00, 2.00, 3.00],
        [2.20, 1.80, 0.70, 0.00, 0.00, 5.00, 1.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70],
        [-4.00, 2.00, 15.00, -2.00, 7.00, 2.00, 11.2],
        [2.20, 1.80, 0.70, 1.00, -3.00, 12.00, -3.00],
        [2.20, 1.80, 4.70, 13.50, 4.30, 1.80, 0.70]]

In [34]:
problem = PeakFindingProblem(initial, grid, directions4)
problem2 = PeakFindingProblem(initial, grid2, directions4)

In [35]:
sumaT=0;
optMejor=0.0
for x in range (100):
    solutions = [problem.value(simulated_annealing(problem)) for i in range(100)]
    count=0
    for i in range(100):
        if solutions[i]>0.0:
            count+=1
    if optMejor <max(solutions):
        optMejor= max(solutions)
    sumaT+=count
print(sumaT/100, "%")
print(optMejor)

39.87 %
11.2


In [36]:
sumaT=0;
optMejor=0.0
for x in range (100):
    solutions = [problem2.value(simulated_annealing(problem2)) for i in range(100)]
    count=0
    for i in range(100):
        if solutions[i]>=5.0:
            count+=1
    if optMejor<max(solutions):
        optMejor= max(solutions)
    sumaT+=count
print(sumaT/100, "%")
print(optMejor)

30.46 %
15.0


In [37]:
solution = problem.value(hill_climbing(problem))
solution

0.0

In [38]:
solution = problem2.value(hill_climbing(problem2))
solution

5.0

En este caso, el algoritmo de escalada genera siempre los mismos vecinos, por lo que siempre devuelve el mismo resultado. En la rejilla 1 da 0 y en la rejilla 2 da 5.

##### Rejilla1 
La media aproximada de veces que se obtiene una solución mejor en el enfriamiento simulado que en el de escalada es del 40% (aproximadamente). Sin embargo, en este ejemplo, el enfriamiento simulado nunca es peor que la escalada , asi qué en un 60% (aproximadamente) obtiene la misma solución. En otras rejillas, puede darse el caso de que el enfriamiento simulado sea peor, pero con una probabilidad bajisima.

##### Rejilla2
En esta rejilla el algoritmo de enfriamiento empeora bastante su rendimiento con respecto al de escalada, pues tan solo en un 30% (aproximadamente) es igual o superior que el de escalada. Sin embargo, en ciertas ejecuciones sí alcanza el máximo global, mientras que en todas las probadas con el algoritmo de escalada, esto nunca se da (siempre obtenemos 5).

#### OBSERVACIONES
Como en el enfriamiento simulado se permite continuar el desarrollo a partir de un vecino peor, es posible evitar los óptimos locales y las mesetas que estancaban al algoritmo de escalada. Así pues, repitiendo una serie de veces el algoritmo, se suele encontrar el óptimo global. Dicha situación es más complicada que se de en el algoritmo de escalada. Esto se pone de manifiesto en los dos ejemplos expuestos anteriormente. 

## Parte 3. Algoritmos genéticos


Se define una clase ProblemaGenetico que incluye los elementos necesarios para la representación de un problema de optimización que se va a resolver con un algoritmo genético. Los elementos son los que hemos visto en clase:

 - genes: lista de genes usados en el genotipo de los estados.
 - longitud_individuos: longitud de los cromosomas
 - decodifica: función de obtiene el fenotipo a partir del genotipo.
 - fitness: función de valoración.
 - muta: función de mutación de un cromosoma 
 - cruza: función de cruce de un par de cromosomas

In [2]:
import random

In [3]:
class ProblemaGenetico(object):
        def __init__(self, genes, fun_dec, fun_muta, fun_cruza, fun_fitness, longitud_individuos):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.longitud_individuos = longitud_individuos
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma)
            return valoracion

En primer lugar vamos a definir una instancia de la clase anterior correspondiente al problema de optimizar (maximizar o minimizar) la función cuadrado x^2 en el conjunto de los números naturales menores que 2^{10}.
Se usa este ejemplo (del que sabemos la solución) para ver todos los elementos y poder observar el comportamiento. 


In [16]:
# Será necesaria la siguiente función que interpreta una lista de 0's y 1's como un número natural:  
# La siguiente función que interpreta una lista de 0's y 1's como
# un número natural:  

def binario_a_decimal(x):
    return sum(b*(2**i) for (i,b) in enumerate(x)) 

In [42]:
list(enumerate([1, 0, 0]))

[(0, 1), (1, 0), (2, 0)]

In [17]:
# En primer luegar usaremos la clase anterior para representar el problema de optimizar (maximizar o minimizar)
# la función cuadrado en el conjunto de los números naturales menores que
# 2^{10}.

# Tenemos que definir funciones de cruce, mutación y fitness para este problema.

def fun_cruzar(cromosoma1, cromosoma2):
    """Cruza los cromosomas por la mitad"""
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2]+cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2]+cromosoma1[l2//2:l1]
    return [cruce1,cruce2]

def fun_mutar(cromosoma,prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    l = len(cromosoma)
    p = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        cromosoma[p] =  (cromosoma[p]+1)%2
    return cromosoma

def fun_fitness_cuad(cromosoma):
    """Función de valoración que eleva al cuadrado el número recibido en binario"""
    n = binario_a_decimal(cromosoma)**2
    return n

cuadrados = ProblemaGenetico([0,1],binario_a_decimal,fun_mutar, fun_cruzar, fun_fitness_cuad,10)

Una vez definida la instancia cuadrados que representa el problema genético, probar alguna de las funciones definidas en la clase anterior, para esta instancia concreta. Por ejemplo:

In [44]:
cuadrados.decodifica([1,0,0,0,1,1,0,0,1,0,1])
# Salida esperada: 1329

1329

In [45]:
cuadrados.fitness([1,0,0,0,1,1,0,0,1,0,1])
# Salida esperada: 1766241

1766241

In [46]:
cuadrados.muta([1,0,0,0,1,1,0,0,1,0,1],0.1)
# Posible salida: [1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

In [47]:
cuadrados.muta([1,0,0,0,1,1,0,0,1,0,1],0.1)
# Posible salida: [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

In [48]:
cuadrados.cruza([1,0,0,0,1,1,0,0,1,0,1],[0,1,1,0,1,0,0,1,1,1])
# Posible salida: [[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

[[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

### Ejercicio 3

   - Definir una función poblacion_inicial(problema_genetico,tamaño), para definir una población inicial de un tamaño dado, para una instancia dada de la clase anterior ProblemaGenetico

sugerencia: usar random.choice

   - Definir una función de cruce que recibe una instancia de Problema_Genetico y una población de padres (supondremos que hay un número par de padres), obtiene la población resultante de cruzarlos de dos en dos (en el orden en que aparecen)

cruza_padres(problema_genetico,padres)

   - Definir la función de mutación que recibe una instancia de Problema_Genetico, una población y una probabilidad de mutación, obtiene la población resultante de aplicar operaciones de mutación a cada individuo llamando a la función muta definida para el problema genético.
   
muta_individuos(problema_genetico, poblacion, prob)

In [ ]:
def poblacion_inicial(problema_genetico, size):
    l = []
    for i in range(size):
        ind = []
        for i in range(problema_genetico.longitud_individuos):
            ind.append(random.choice(problema_genetico.genes))
        l.append(ind) 
    return l

In [50]:
poblacion_inicial(cuadrados,10)

[[1, 1, 0, 1, 1, 0, 0, 0, 1, 1],
 [1, 0, 0, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 1, 0, 1, 0, 1, 0],
 [0, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 1, 0, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]]

In [9]:
def cruza_padres(problema_genetico,padres):
    l = []
    i = 0
    while i < len(padres)/2:
        u = fun_cruzar(padres[2*i], padres[2*i+1])
        l.append(u[0])
        l.append(u[1])
        i += 1
    return l

In [52]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

cruza_padres(cuadrados,p1)
# Posible salida
# [[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
#  [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#  [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#  [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
#  [0, 1, 1, 1, 1, 0, 1, 1, 0, 1],
#  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]]

[[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 0]]

In [10]:
def muta_individuos(problema_genetico, poblacion, prob):
    for i in range(len(poblacion)):
        poblacion[i] = problema_genetico.muta(poblacion[i], prob)
    return poblacion

In [54]:
muta_individuos(cuadrados, p1, 0.5)
# Posible salida:
#  [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#   [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
#   [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
#   [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#   [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#   [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

[[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

In [55]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

In [56]:
muta_individuos(cuadrados, p1, 0.5)

[[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

### Ejercicio 4

Se pide definir una función de selección mediante torneo de n individuos de una población.  
La función recibe como entrada:
 - una instancia de la clase ProblemaGenetico
 - una población
 - el número n de individuos que vamos a seleccionar
 - el número k de participantes en el torneo
 - un valor opt que puede ser o la función max o la función min (dependiendo de si el problema es de maximización o de minimización, resp.).

seleccion\_por\_torneo(problema_genetico,poblacion,n,k,opt) 

INDICACIÓN: Usar random.sample para seleccionar k elementos de una secuencia. 
Por ejemplo, random.sample(population=[2,5,7,8,9], k=3) devuelve [7,5,8]. 

In [11]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
    return seleccionados  

In [58]:
#Ejemplo
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados,8),3,6,max)
# Posible salida: [[1, 1, 1, 1, 1, 0, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1, 1, 0, 1]]


[[1, 0, 1, 0, 1, 1, 1, 0, 0, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 0, 1],
 [1, 0, 1, 0, 1, 0, 1, 1, 1, 0]]

In [59]:
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados,8),3,6,min)
# [[0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0, 1, 0, 1, 0]]

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [12]:
# La siguiente función implementa una posibilidad para el algoritmo genético completo: 
# inicializa t = 0 
# Generar y evaluar la Población P(t)
# Mientras no hemos llegado al número de generaciones fijado:  t < nGen
#    P1 = Selección por torneo de (1-size)·p individuos de P(t)
#    P2 = Selección por torneo de (size·p) individuos de P(t)
#    P3 = Aplicar cruce en la población P2
#    P4 = Union de P1 y P3
#    P(t+1) := Aplicar mutación P4 
#    Evalua la población P(t+1) 
#    t:= t+1
        
# Sus argumentos son:
# problema_genetico: una instancia de la clase ProblemaGenetico con la representación adecuada del problema de optimización 
# que se quiere resolver.
# k: número de participantes en los torneos de selección.
# opt: max ó min, dependiendo si el problema es de maximización o de minimización. 
# nGen: número de generaciones (que se usa como condición de terminación)
# size: número de individuos en cada generación
# prop_cruce: proporción del total de la población que serán padres. 
# prob_mutación: probabilidad de realizar una mutación de un gen.

def algoritmo_genetico(problema_genetico, k, opt, ngen, size, prop_cruces, prob_mutar):
    poblacion= poblacion_inicial(problema_genetico, size)
    n_padres = round(size*prop_cruces)
    n_padres = int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion = nueva_generacion(problema_genetico, k, opt, poblacion, n_padres, n_directos, prob_mutar)
        

    mejor_cr = opt(poblacion, key=problema_genetico.fitness)
    mejor = problema_genetico.decodifica(mejor_cr)
    return (mejor, problema_genetico.fitness(mejor_cr)) 


Necesitarás definir la función auxiliar nueva_generacion(problema_genetico,poblacion,n_padres,n_directos,prob_mutar) que dada una población calcula la siguiente generación.

In [13]:
#Definir la función nueva_generacion

def nueva_generacion(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar):
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k,opt)
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico, padres2)
    generacion = padres1+cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

### Ejercicio 5.  Ejecutar el algoritmo genético anterior, para resolver el problema anterior (tanto en minimización como en maximización).  

Hacer una valoración de resultados y comentarios sobre el comportamiento del algoritmo. 
En la resolución del problema hay que tener en cuenta que el algoritmo genético devuelve un par con el mejor fenotipo encontrado y su valoración.

In [16]:
cont = 0
for _ in range (100):
    x = algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.1)
    if x == (0,0):
        cont += 1
cont
# Salida esperada: (0, 0)

54

In [17]:
cont = 0
for _ in range (100):
    x = algoritmo_genetico(cuadrados,3,max,20,10,0.7,0.1)
    if x == (1023, 1046529):
        cont += 1
cont
# Salida esperada: (1023, 1046529)

55

### OBSERVACIONES
En cuanto al funcionamiento interno:
1. Método de selección: Se ha elegido la selección por torneo, que combina lo mejor del método elitista (se escogen los mejores individuos) y del método de la ruleta (interviene la aleatoriedad), que permite aumentar el árbol de soluciones explorado.
2. Mutación: Se introduce la mutación de la población con una pequeña probabilidad (0.1), para obtener soluciones más variadas, que al final pueden llegar a ser las mejores. El elitismo puro y duro no siempre acaba encontrando la solución óptima; y la variedad puede ayudar a llegar a ella.
3. Cruce: El algoritmo de cruce escogido es de tipo destructivo, pues los descendientes de los cruces siempre van a estar en la nueva generación, independientemente de su fitness (se tiene en cuenta en la selección de los padres). Sin embargo, algunos individuos de la generación antigua también pasan a la nueva (son los padres1). Con esto se pretende que algunos cromosomas eficaces no se pierdan por el camino de los cruces y las mutaciones. Además, es un cruce de un punto (en la mitad de los cromosomas).

En cuanto a los resultados:
1. Vemos que, de media, nos sale 50-60% (aprox.) de veces en las que encontramos la solución óptima. Esto está bien pues, como ya sabemos, el algortimo genético no garantiza que se llegue a esta solución óptima. 
2. La causa de que muchas veces no alcancemos la solución óptima se debe a la selección por torneo. Aunque elijamos los mejores participantes en cada torneo, la elección de los participantes es totalmente aleatoria y esto puede causar que los participantes no sean "muy buenos" pudiendo hacer que las próximas generaciones no sean mucho mejores (incluso podrían ser peores, aunque en este caso es raro pues las nuevas generaciones tendrán más individuos que la anterior y la mitad de estos serán idénticos a algunos de la generación anterior). Además, también influye la mutación (que es buena para el problema pues hace que la población y nuevas generaciones tengan variedad de individuos) que hace que unos descendientes que hayan sido buenos muten haciendo que el resultado sea una incógnita (el hijo mutado podría tener un fitness similares o mejor, pero también podría ser peor)

##  El problema de la mochila 

Se plantea el típico problema de la mochila en el que dados n objetos de pesos conocidos pi y valor vi (i=1,...,n) hay que elegir cuáles se meten en una mochila que soporta un peso P máximo. La selección debe hacerse de forma que se máximice el valor de los objetos introducidos sin superar el peso máximo.

### Ejercicio 6
Se pide definir la representación del problema de la mochila usando genes [0,1] y longitud de los individuos n.

Los valores 1 ó 0 representan, respectivamente, si el objeto se introduce o no en la mochila Tomados de izquerda a derecha, a partir del primero que no cabe, se consideran  todos fuera de la mochila,independientemente del gen en su posición. De esta manera, todos los individuos representan candidatos válidos.

El numero de objetos n determina la longitud de los individuos de la población.
En primer lugar es necesario definir una función de decodificación de la mochila que recibe como entrada:
* un cromosoma (en este caso, una lista de 0s y 1s, de longitud igual a n_objetos) 
* n: número total de objetos de la mochila
* pesos: una lista con los pesos de los objetos
* capacidad: peso máximo de la mochila.
La función decodifica recibe (cromosoma, n, pesos, capacidad) y devuelve una lista de 0s y 1s que indique qué objetos están en la mochila y cuáles no (el objeto i está en la mochila si y sólo si en la posición i-ésima de la lista hay un 1). Esta lista se obtendrá a partir del cromosoma, pero teniendo en cuenta que a partir del primer objeto que no quepa, éste y los siguientes se consideran fuera de la mochila, independientemente del valor que haya en su correspondiente posición de cromosoma. 

In [3]:
class ProblemaGeneticoMochila(object):
        def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness, longitud_individuos, pesos, valores, capacidad):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.longitud_individuos = longitud_individuos
            self.pesos=pesos
            self.valores=valores
            self.capacidad=capacidad
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo,self.longitud_individuos,self.pesos,self.capacidad)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma,self.longitud_individuos,self.pesos,self.capacidad,self.valores)
            return valoracion

In [4]:
def decodifica_mochila(cromosoma, n, pesos, capacidad):
    peso_en_mochila = 0
    l = []
    for i in range(n):
        if cromosoma[i] == 1 and peso_en_mochila + pesos[i] <= capacidad:
            l.append(1)
            peso_en_mochila += pesos[i]
        elif cromosoma[i] == 0 or peso_en_mochila + pesos[i] > capacidad:
            l.append(0)
    return l

In [20]:
decodifica_mochila([1,1,1,1,1], 5, [2,2,4,5,1], 5)

[1, 1, 0, 0, 1]

Para definir la función de evaluación (fitness) necesitamos calcular el valor total de los objetos que están dentro de la mochila que representa el cromosoma según la codificación utilizada en la función anterior. 

Se pide la función fitness (cromosoma, n_objetos, pesos, capacidad, valores) donde los parámetros son los mismos que en la función anterior, y valores es la lista de los valores de cada objeto

fitness(cromosoma, n_objetos, pesos, capacidad, valores)

Ejemplo de uso:
   fitness([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])
   7

In [5]:
def fitness_mochila(cromosoma, n_objetos, pesos, capacidad, valores):
    valor = 0
    cValido = decodifica_mochila(cromosoma,n_objetos,pesos,capacidad)
    for i in range(n_objetos):
        valor += (cValido[i]*valores[i])
    return valor

In [22]:
fitness_mochila([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])

7

Damos tres instancias concretas del problema de la mochila. Damos también sus soluciones optimas, para que se puedan comparar con los resultados obtenidos por el algoritmo genético:

##### REPRESENTACIÓN
Se reutilizan las funciones mutar, cruzar y selección por torneo del problema de maximizar x^2

In [23]:
# Problema de la mochila 1:
# 10 objetos, peso máximo 165
pesos1 = [23,31,29,44,53,38,63,85,89,82]
valores1 = [92,57,49,68,60,43,67,84,87,72]
# Solución óptima= [1,1,1,1,0,1,0,0,0,0], con valor 309

In [24]:
# Problema de la mochila 2:
# 15 objetos, peso máximo 750

pesos2 = [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120]
valores2 = [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240]

# Solución óptima= [1,0,1,0,1,0,1,1,1,0,0,0,0,1,1] con valor 1458

In [6]:
# Problema de la mochila 3:
# 24 objetos, peso máximo 6404180
pesos3 = [382745,799601,909247,729069,467902, 44328,
       34610,698150,823460,903959,853665,551830,610856,
       670702,488960,951111,323046,446298,931161, 31385,496951,264724,224916,169684]
valores3 = [825594,1677009,1676628,1523970, 943972,  97426,
       69666,1296457,1679693,1902996,
       1844992,1049289,1252836,1319836, 953277,2067538, 675367,
       853655,1826027, 65731, 901489, 577243, 466257, 369261]

# Solución óptima= [1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1] con valoración 13549094

### Ejercicio 7

Definir variables m1g, m2g y m3g, referenciando a instancias de Problema_Genetico que correspondan, respectivamente, a los problemas de la mochila anteriores. Resuelve los problemas y comentar los resultados obtenidos en cuanto a eficiencia y calidad de los resultados obtenidos.

Algunas de las salidas posibles variando los parámetros.

In [26]:
# >>> algoritmo_genetico_t(m1g,3,max,100,50,0.8,0.05)
# ([1, 1, 1, 1, 0, 1, 0, 0, 0, 0], 309)

# >>> algoritmo_genetico_t(m2g,3,max,100,50,0.8,0.05)
# ([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0], 1444)
# >>> algoritmo_genetico_t(m2g,3,max,200,100,0.8,0.05)
# ([0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0], 1439)
# >>> algoritmo_genetico_t(m2g,3,max,200,100,0.8,0.05)
# ([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1], 1458)

# >>> algoritmo_genetico_t(m3g,5,max,400,200,0.75,0.1)
# ([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], 13518963)
# >>> algoritmo_genetico_t(m3g,4,max,600,200,0.75,0.1)
# ([1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0], 13524340)
# >>> algoritmo_genetico_t(m3g,4,max,1000,200,0.75,0.1)
# ([1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 13449995)
# >>> algoritmo_genetico_t(m3g,3,max,1000,100,0.75,0.1)
# ([1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 13412953)
# >>> algoritmo_genetico_t(m3g,3,max,2000,100,0.75,0.1)
# ([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 13366296)
# >>> algoritmo_genetico_t(m3g,6,max,2000,100,0.75,0.1)
# ([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1], 13549094)

In [27]:
m1g = ProblemaGeneticoMochila([0,1],decodifica_mochila,fun_mutar, fun_cruzar, fitness_mochila, 10, pesos1, valores1, 165)
cont = 0
for i in range (100):
    if algoritmo_genetico(m1g,3,max,100,50,0.8,0.05)[1]<309:
        cont += 1
print("El {num}% (aproximadamente) no encuentra la solución óptima".format(num=cont))


El 5% (aproximadamente) no encuentra la solución óptima


In [28]:
m2g = ProblemaGeneticoMochila([0,1],decodifica_mochila, fun_mutar, fun_cruzar, fitness_mochila,15,pesos2,valores2,750)
cont = 0
for i in range (1000):
    if algoritmo_genetico(m2g,3,max,100,50,0.8,0.05)[1] < 1458: #1438
        cont += 1
print("El {num}% (aproximadamente) no encuentra la solución óptima, en la primera configuración".format(num = cont/10))
      
cont = 0
for i in range (1000):
    if algoritmo_genetico(m2g,3,max,200,100,0.8,0.05)[1] < 1458:
        cont += 1
print("El {num}% (aproximadamente) no encuentra la solución óptima, en la segunda configuración".format(num = cont/10))


El 99.4% (aproximadamente) no encuentra la solución óptima, en la primera configuración
El 97.2% (aproximadamente) no encuentra la solución óptima, en la segunda configuración


In [20]:
m3g = ProblemaGeneticoMochila([0,1],decodifica_mochila,fun_mutar, fun_cruzar, fitness_mochila,24,pesos3,valores3,6404180)
cont = 0
for i in range (100):
    if algoritmo_genetico(m3g,5,max,400,200,0.75,0.1)[1] < 13549094:  #13400000
        cont += 1
print("El {num}% (aproximadamente) no encuentra la solución óptima, en la primera configuración".format(num = cont))
      
cont = 0
for i in range (100):
    if algoritmo_genetico(m3g,5,max,1000,200,0.8,0.05)[1] < 13549094:
        cont += 1
print("El {num}% (aproximadamente) no encuentra la solución óptima, en la segunda configuración".format(num = cont))

El 95% (aproximadamente) no encuentra la solución óptima, en la primera configuración
El 98% (aproximadamente) no encuentra la solución óptima, en la segunda configuración


### OBSERVACIONES
1. En el primer caso vemos que la solución óptima se encuentra en un 95% (aprox.) de los casos y esto no es raro pues la probabildad de cruce es alta 80% y la de mutación muy baja 5%. Esto es así, pues en principio las siguientes generaciones serán mejores que las anteriores y como el cruce es alto y la mutación es baja se favorece este hecho. 
2. Sin embargo, en los siguiente ejemplos vemos que casi el 100% de los casos no encuentra solución óptima (en el 99% y 98% aproximadamente en las respectivas configuraciones). ¿Esto a qué se debe? Este hecho no es malo, de hecho es normal. Lo que pasa en los algoritmos genéticos (sobre todo cuando la longitud de individuos y tamaño de la población son grandes) es que se genera un árbol de exploración muy muy grande y es muy difícil llegar a la solución exactamente óptima. En este tipo de algoritmos se obtienen muchas soluciones subóptimas, cerca de la óptima, pero sin llegar a ser esta. Esto lo hemos comprobado modificando "ligeramente" el valor de fitness máximo. Haciendo esto, hemos verificado que se obtiene un porcentaje de soluciones subóptimas altísimo: (por ejemplo en el caso 2 se obtiene pasando de 1458 a 1438 el 75% de casos verifican este fitness). Además, reduciendo un poco más esta función de fitness (de 1458 a 1410 por ejemplo) se obtiene el 100% de los casos que tienen en ese fitness!!!

### Ejercicio 8
Resolver mediante una configuración de un algoritmo genético el problema del cuadrado mágico que consiste en colocar en un cuadrado n × n los números naturales de 1 a n^2, 
de tal manera que las filas, las columnas y las diagonales principales sumen los mismo. 
Ejemplo: una solucion para n= 3
    
    4 3 8
    9 5 1
    2 7 6
    
- Dimension del cuadrado: N
- Suma común: SUMA=(N·(N^2 + 1))/2
    
    Comenta el resultado y el rendimiento del algoritmo para distintos parámetros.
    

    
    

In [11]:
from itertools import permutations
import math
import random

def poblacion_inicial_cm(problema_genetico, size):
    l = []
    for i in range(size):
        l2 = list(permutations(range(1,problema_genetico.longitud_individuos+1)))
        x = random.randint(0, math.factorial(problema_genetico.longitud_individuos) - 1)
        l.append(l2[x])  
    return l

def cruza_padres_cm(problema_genetico, padres):
    l = []
    i = 0
    while i < len(padres)/2:
        u = problema_genetico.cruza(padres[2*i], padres[2*i+1])
        l.append(u[0])
        l.append(u[1])
        i += 1
    return l

def seleccion_por_torneo_cm(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
    return seleccionados  

def muta_individuos_cm(problema_genetico, poblacion, prob):
    for i in range(len(poblacion)):
        poblacion[i]=problema_genetico.muta(poblacion[i],prob)
    return poblacion

def nueva_generacion_cm(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar):
    padres1 = seleccion_por_torneo_cm(problema_genetico, poblacion, n_directos, k,opt)
    padres2 = seleccion_por_torneo_cm(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres_cm(problema_genetico,padres2)
    generacion = padres1+cruces
    resultado_mutaciones = muta_individuos_cm(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones


def algoritmo_genetico_cm(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar):
    poblacion= poblacion_inicial_cm(problema_genetico,size)
    n_padres= round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion= nueva_generacion_cm(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)
        
    mejor_cr = opt(poblacion, key=problema_genetico.fitness)
    mejor = problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr))



In [12]:
class ProblemaGenetico(object):
        def __init__(self, genes, fun_dec, fun_muta, fun_cruza, fun_fitness, longitud_individuos):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.longitud_individuos = longitud_individuos
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma)
            return valoracion

In [13]:
def decodifica_cm(cromosoma):
    return cromosoma

#Implementado el algoritmo PMX
def cruzar_cm1(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    a = random.randint(0,l1-l1//2)
    b = random.randint(a+1,l1-2)
    la1 = list(cromosoma1[0:a])
    la2 = list(cromosoma1[a:b+1])
    la3 = list(cromosoma1[b+1:l1])
    lb1 = list(cromosoma2[0:a])
    lb2 = list(cromosoma2[a:b+1])
    lb3 = list(cromosoma2[b+1:l1])
    #Construir la1:
    for i in range(a):
        k=i
        while(cromosoma2[k] in la2):
            k=cromosoma1.index(cromosoma2[k])
        la1[i]=cromosoma2[k]
    #Construir la3:
    for j in range(l1 - (b+1)):
        k=j + b+1
        while(cromosoma2[k] in la2):
            k=cromosoma1.index(cromosoma2[k])
        la3[j]=cromosoma2[k]
        
    cruce1=la1 + la2 + la3
    #Construir lb1:
    for i in range(a):
        k=i
        while(cromosoma1[k] in lb2):
            k=cromosoma2.index(cromosoma1[k])
        lb1[i]=cromosoma1[k]
   
    #Construir lb3:
    for j in range(l1 - (b+1)):
        k=j + b+1
        while(cromosoma1[k] in lb2):
            k=cromosoma2.index(cromosoma1[k])
        lb3[j]=cromosoma1[k]
    
    cruce2=lb1 + lb2 + lb3
    return [tuple(cruce1),tuple(cruce2)]

#Implementado el método de inversión
def mutar_cm(cromosoma,prob):
    if prob > random.uniform(0,1):
        l=len(cromosoma)
        i=random.randint(0,l-2)
        j=random.randint(i+1,l-1)
        cromosoma=cromosoma[0:i]+tuple(reversed(cromosoma[i:j]))+cromosoma[j:l]
    return cromosoma

#Implementamos el algoritmo: cruce de orden
def cruzar_cm2(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    a = random.randint(0,l1-l1//2)
    b = random.randint(a+1,l1-2)
    listaIntc1=list(cromosoma1[a:b+1])
    listaIntc2=list(cromosoma2[a:b+1])
    cruce1=list(cromosoma1)
    cruce2=list(cromosoma2)
    ind1=ind2=b+1
    for i in range (l1):
        j=(i + b +1)%l1
        if cromosoma2[j] not in listaIntc1:
            cruce1[ind1]=cromosoma2[j]
            ind1=(ind1+1)%l1
        if cromosoma1[j] not in listaIntc2:
            cruce2[ind2]=cromosoma1[j]
            ind2=(ind2+1)%l1
    return [tuple(cruce1),tuple(cruce2)]

#Minimizar la máxima diferencia de alguna de las sumas con el valor N(N^2+1)/2 =>
#El fitness es la máxima diferencia
def fitness_cm(cromosoma):
    maxdif=0
    n=int (math.sqrt(len(cromosoma)))
    sumaBuscada=n*(n**2+1)/2
    suma=0
    #Filas
    for i in range(n):
        suma=0
        for k in range(n):
            suma+=cromosoma[n*i + k]
        maxdif=max(maxdif,abs(suma-sumaBuscada))
        
    #Columnas
    for j in range(n):
        suma=0
        for k in range(n):
            suma+=cromosoma[j + n*k]
        maxdif=max(maxdif,abs(suma-sumaBuscada))
    
    #Diagonal superior
    suma=0
    for i in range(n):
        suma+=cromosoma[i*(n+1)]
    maxdif=max(maxdif,abs(suma-sumaBuscada))
    
    #Diagonal inferior
    suma=0
    for i in range(n):
        suma+=cromosoma[(n-1)*(i+1)]
    maxdif=max(maxdif,abs(suma-sumaBuscada))
    return maxdif


In [14]:
n=3

In [16]:
#Utilizando PMX
cuadrado_magico1 = ProblemaGenetico([x for x in range(1,n**2+1)], decodifica_cm, mutar_cm, cruzar_cm1, fitness_cm, n**2)
cont = 0
for i in range(100):
    x = algoritmo_genetico_cm(cuadrado_magico1,5,min,100,50,0.7,0.1)[1]
    if x == 0:
        cont += 1
print("El {num}% (aproximadamente) encuentra la solución óptima".format(num=cont))

El 19% (aproximadamente) encuentra la solución óptima


In [18]:
#Utilizando cruce de orden
cuadrado_magico1 = ProblemaGenetico([x for x in range(1,n**2+1)], decodifica_cm, mutar_cm, cruzar_cm2, fitness_cm, n**2)
cont = 0
for i in range(100):
    x = algoritmo_genetico_cm(cuadrado_magico1,5,min,100,50,0.7,0.1)[1]
    if x == 0:
        cont += 1
print("El {num}% (aproximadamente) encuentra la solución óptima".format(num=cont))

El 6% (aproximadamente) encuentra la solución óptima


### OBSERVACIONES
La representación del problema se basa en permutaciones de los n^2 números. Por lo tanto, los genes no son un conjunto de bits, si no un único natural comprendido entre 1 y n^2. En cuanto a los resultados obtenidos:
En la primera resolución hemos utilizado el PMX y en la segunda el cruce de orden. Se observa que para este problema, el cruce PMX es más eficaz. Esto se puede deber a que, normalmente, el algoritmo PMX "cambia menos" los individuos de la siguiente generación.

Vemos que los números han de seguir un patrón para poder obtener el resultado (todos los datos los exponemos para n=3):
1. El 9 solo puede sumar 15 con 2 combinaciones {1,5} y {2,4}. Esto significa que el 9 solo puede ocupar las casillas que intervengan en dos sumas (solo hay cuatro posibilidades, que serían ARRIBA,ABAJO,DERECHA,IZQUIERDA desde el medio (evitando las esquinas)).
2. Al 7 le ocurre lo mismo. Esto significa que ya están asignadas esas cuatro posiciones, pues al 1 y al 3 les sucede lo mismo: {9,1} enfrentados y {7,3}
3. El 5 ha de ocupar la posición central, pues es el único que aparece en cuatro posibles combinaciones.
4. Viendo los números anteriores, ya tendríamos las posibles casillas para el resto.

Por lo tanto, si obtenemos algunos de estos patrones, es más probable que el algoritmo de cruce de PMX los desordene menos. De esta forma, se consigue que si obtenemos una solución o alguno de los patrones anteriores (que se acercan a conseguir una solución) se desordenen menos en la siguiente generación y tengamos más posibilidad de llegar a una solución en esta.

### Ejercicio 9
Se quiere organizar una **jornada de la informática** en la Facultad. 
El Vicedecano necesita organizar el horario y la asignación de profesores para n actividades temáticas, a_1, . . . , a_n.  Dispone de m profesores p_1, . . . , p_m con distintos niveles de cualificación. 
No todos los profesores tienen los conocimientos para encargarse de todas las actividades. 
El mismo profesor pi puede encargarse de distintas actividades (que se le remunerarán convenientemente) pero no podemos asignar al mismo profesor dos actividades que se impartan simultáneamente. 
Cada actividad puede durar un tiempo diferente dependiendo del profesor que la organice. Si un profesor p_i puede realizar la actividad a_j, entonces denotaremos por t_ij el tiempo que pi tardará en realizar la actividad a_j. 
Existen dependencias entre los contenidos de las actividades lo que obliga a incluir algunas restricciones temporales, es decir, algunas actividades hay que realizarlas cuando otras ya hayan terminado. Esto es, tenemos un conjunto de restricciones que indican que ai se debe realizar antes de a_j. Por ejemplo, El taller de iniciación a Scratch se debe realizar antes que el laboratorio de desarrollo de videojuegos con Scratch.   
Se pide dar una solución al problema de decidir, para cada actividad, qué profesor la realizará y en qué momento se debe ofertar la actividad para que, al ﬁnal de la jornada, se puedan realizar todas las actividades con cuidado de cumplir los prerrequisitos (restricciones temporales) y el conjunto de actividades se realice en el menor tiempo posible.

In [23]:
import math
import random
def poblacion_inicial_jI(problema_genetico, size):
    l = []
    for i in range(size):
        ind = []
        for i in range(problema_genetico.longitud_individuos):
            ind.append(random.choice(problema_genetico.genes))
        l.append(tuple(ind)) 
    return l

def cruza_padres_jI(problema_genetico,padres):
    l = []
    i = 0
    while i < len(padres)/2:
        u = problema_genetico.cruza(padres[2*i], padres[2*i+1])
        l.append(u[0])
        l.append(u[1])
        i += 1
    return l

def seleccion_por_torneo_jI(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
    return seleccionados  

def muta_individuos_jI(problema_genetico, poblacion, prob_mutar1,prob_mutar2,duracionMax,m):
    for i in range(len(poblacion)):
        poblacion[i]=mutar_jI(poblacion[i],prob_mutar1,prob_mutar2,duracionMax,m)
    return poblacion

def nueva_generacion_jI(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar1,prob_mutar2,duracionMax,m):
    padres1 = seleccion_por_torneo_jI(problema_genetico, poblacion, n_directos, k,opt)
    padres2 = seleccion_por_torneo_jI(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres_jI(problema_genetico,padres2)
    generacion = padres1+cruces
    resultado_mutaciones = muta_individuos_jI(problema_genetico, generacion, prob_mutar1,prob_mutar2,duracionMax,m)
    return resultado_mutaciones


def algoritmo_genetico(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar1,prob_mutar2,duracionMax,m):
    poblacion= poblacion_inicial_jI(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion= nueva_generacion_jI(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar1,prob_mutar2,duracionMax,m)
    mejor_cr= opt(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr))


In [24]:
n=4
m=4
tiempos=[[60,0,90,0],[0,70,0,80],[0,0,0,0],[0,0,0,0]]
#Lista de restricciones temporales
restricciones=[[],[],[],[]]

def decodifica_jI(cromosoma):
    c=[]
    l=len(cromosoma)
    for i in range(l):
        a=cromosoma[i][0]
        b=cromosoma[i][1]*10
        c.append([a,b])
    return tuple(c)

#Implementado el algoritmo: cruce de dos puntos
def cruzar_jI(cromosoma1, cromosoma2):
    n= len(cromosoma1)
    a = random.randint(0,n//2)
    b=random.randint(a+1,n-1)
    cruce1=cromosoma1[0:a] + cromosoma2[a:b+1] + cromosoma1[b+1:n]
    cruce2=cromosoma2[0:a] + cromosoma1[a:b+1] + cromosoma2[b+1:n]
    return [tuple(cruce1),tuple(cruce2)]

#Implementado el método de inversión
def mutar_jI(cromosoma,prob_profesor,prob_tiempo,duracionMax,m):
    n=len(cromosoma)
    c=list(cromosoma)
    if prob_profesor > random.uniform(0,1):
        for k in range(2):
            i=random.randint(0,n-1)
            prof=random.randint(0,m-1)
            c[i][0]=prof
    if prob_tiempo>random.uniform(0,1):
        for k in range(2):
            j=random.randint(0,n-1)
            tiempo=random.randint(0,duracionMax-1)
            c[j][1]=tiempo
    return tuple(c)

def tiempoMax(cromosoma,tiempos):
    maximo=0
    for i in range(len(cromosoma)):
        maximo=max(cromosoma[i][1] + tiempos[cromosoma[i][0]][i],maximo)
    return maximo

def numRestriccionTemporales(cromosoma,tiempos,restricciones):
    cont=0
    n=len(cromosoma)
    for i in range(n):
        for j in range(len(restricciones[i])):
            if cromosoma[j][1] + tiempos[cromosoma[j][0]][j]>cromosoma[i][1]:
                cont+=1
    return cont

def numRestriccionProfesores(cromosoma,tiempos):
    cont=0
    m=len(tiempos)
    n=len(cromosoma)
    intervalos = []
    for i in range(m):
        intervalos.append([])
    for i in range(n):
        profesor=cromosoma[i][0]
        if tiempos[profesor][i]==0:
            cont+=1
        else:
            lInterv=intervalos[profesor]
            tInit=cromosoma[i][1]
            tFin=tInit+tiempos[profesor][i]
            lInterv.append((tInit,tFin))
            for j in range(len(lInterv)):
                if tInit>lInterv[j][0] and tInit<lInterv[j][1]:
                    cont+=1
                elif tFin>lInterv[j][0] and tFin<lInterv[j][1]:
                    cont+=1
                
    return cont
    
def fitness_jI(cromosoma):
    cromosomaDecod=decodifica_jI(cromosoma)
    tM=tiempoMax(cromosomaDecod,tiempos)
    r1=numRestriccionTemporales(cromosoma,tiempos,restricciones)
    r2=numRestriccionProfesores(cromosoma,tiempos)
    return tM + 220*r1 + 220*r2


In [25]:
def precalculo(t):
    tot=0
    for j in range(len(t[0])):
        maximo=0
        for i in range(len(t)):
            maximo=max(maximo,t[i][j])
        if maximo==0:
            return -1
        tot+=maximo
    return tot

In [26]:
#Dividimos /10 para obtener tiempos de inicio multiplos de 10 minutos. En el decodifica ya se multiplica de nuevo *10
duracionMax=precalculo(tiempos)//10
if duracionMax<0:
    print("El caso de prueba no tiene solución")
else:
    cont=0
    genes=[[x,y] for x in range(len(tiempos)) for y in range (duracionMax+1)] #dupla:(profesor,tiempo_inicio)
    jornadaInformatica=ProblemaGenetico(genes,decodifica_jI,mutar_jI, cruzar_jI, fitness_jI, n)
    for i in range(10):
        sol=algoritmo_genetico(jornadaInformatica,20,min,1000,100,0.7,0.4,0.3,duracionMax,m)
        if(sol[1]<600):
            cont+=1
    print(cont)

2


In [27]:
for i in range(10):
    sol=algoritmo_genetico(jornadaInformatica,50,min,500,250,0.7,0.4,0.3,duracionMax,m)
    if(sol[1]<600):
        cont+=1
print(cont)

3


#### RESOLUCION 
Hemos introducido una restricción a nuestro problema: las activividades comienzan en tiempos múltiplo de 10.
1. Los genes son una dupla (p_i,t_i) donde p_i es el profesor que realiza la actividad i y t_i es el tiempo de inicio de la actividad i. Por lo tanto, un cromosoma está compuesto por n genes, siendo n el número de actividades.
2. Fitness: Calculamos el tiempo máximo que va a ser necesario para realizar las n actividades según esten organizadas en el cromosoma. A dicho tiempo añadimos las dos penalizaciones posibles: actividades impartidas por profesores que las desconocen; y actividades que no verifican el orden impuesto en las precondiciones. Para que tengan efecto en el fitness, las multiplicamos por un valor de penalización, que es variable en función de los datos del enunciado. Por lo tanto, se pretende minimizar el fitness para encontrar el mejor individuo posible.
3. Cruce: Cruce de dos puntos
4. Mutar: Tenemos dos probabilidades de mutar, una para los profesores y otra para los tiempos. Es posible mutar un profesor de una actividad y un tiempo de inicio de otra diferente.
5. Decodificar: Los tiempos de inicio son múltiplos de 10 (en teoría minutos). Por ello, en el gen t_i en realidad es t_i%10, y es en el decodifica cuando convertimos a los tiempos reales.
6. Selección y nueva generación: mantenemos el mismo código que en el ejercicio 8 y 7

### OBSERVACIONES
El algoritmo genético es útil si buscamos obtener rápidas soluciones, que no sean óptimas pero que no esten muy lejos, es decir, que sean subóptimas. Este hecho se pone de manifiesto en este problema, pues el objetivo es obtener un horario lo más ajustado posible, teniendo en cuenta de que en la realidad, perder 3 minutos no es realmente importante, si no cumplir las restricciones. De vez en cuando el algoritmo ofrece una solución que no verifica todas las restricciones, pero sí la mayoría de ellas.Por ello sigue siendo eficaz, ya que el usuario puede modificar a mano la solución ya obtenida para verificar todas las condiciones.

En las exploraciones que hemos realizado, observamos que a partir de 500 generaciones, el algoritmo empieza a dar soluciones subóptimas aceptables. Antes de las 500 muestra resultados que difieren mucho de nuestros intereses. Esto no ha de extrañarnos, pues en los algoritmos genéticos, se trata de descubrir la configuración que nos aproxime a una zona próxima a la óptima. En términos de gráficos, en esta zona el algoritmo genético no experimenta grandes mejoras aun aumentando considerablemente el número de individuos o el número de generaciones.